In [16]:
import json
import random
import string
import os
import pandas as pd
import re

import nltk
import numpy as np
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

nltk.download("punkt")
nltk.download("wordnet")
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\adiha\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\adiha\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\adiha\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

## Creating Question Pattern

In [3]:
class Modeling_rule_based:
    # get product name manual
    ## filter product name, perlu di regex lagi
    def get_product(self, p):
        p = p[p.find('*'):len(p)]
        p = p[:p.find('\n')]
        p = p.lower()
        p = re.sub("[^0-9a-z]+", " ", p)

        replacements = {
            '(?<=\s|[0-9])+(g|gr)+(?=\s|\.|$)' : 'gram',
            '(?<=\s|[0-9])+mg+(?=\s|\.|$)' : 'miligram',
            '(?<=\s|[0-9])+ml+(?=\s|\.|$)' : 'mililiter',
            '(?<=\s|[0-9])+l+(?=\s|\.|$)' : 'liter'
        }

        for key, value in replacements.items():
            p = re.sub(key, value, p)

        # p = re.sub("gr",  "gram", p)
        # p = re.sub("ml", "mililiter", p)
        return p

    # create question manual
    ## keyword berdasarkan huruf terbanyak yang muncul
    def create_questions(self, c, q):
        list_q = []
        if c == "product_info":
            q1 = q 
            q2 = "kegunaan " + q
            q3 = "penggunaan " + q
            q4 = "cara pakai " + q
            q5 = "bagaimana menggunakan " + q
            q6 = "cara menggunakan " + q
            q7 = "cara penyimpanan " + q
            q8 = "bagaimana menyimpan " + q
            q9 = "cara menyimpan " + q
            q10 = "apa itu " + q
            q11 = "harga " + q
            q12 = "berapa biaya " + q
            list_q = [q1, q2, q3, q4, q5, q6, q7, q8, q9, q10, q11, q12]
        elif c == "treatment_info":
            q1 = q
            q2 = "apa itu " + q
            q3 = "manfaat " + q
            q4 = "manfaat menggunakan" + q
            q5 = "keuntungan menggunakan " + q
            q6 = "prosedur " + q
            q7 = "bagaimana pengerjaan " + q
            q8 = "bagaimana cara kerja " + q
            q9 = "durasi " + q
            q10 = "berapa lama " + q
            q11 = "harga " + q
            q12 = "berapa biaya " + q
            list_q = [q1, q2, q3, q4, q5, q6, q7, q8, q9, q10, q11, q12]
        
        return list_q

In [4]:
model = Modeling_rule_based()

In [5]:
os.chdir("E:\\Documents\\Coding\\Notebooks\\Chatbot\\apotek")
print(os.getcwd())

E:\Documents\Coding\Notebooks\Chatbot\apotek


In [6]:
df = pd.read_csv("dataset_transformation - dataset_transformation.csv")

In [7]:
df.head()

,Content ID,Answer 1,answer_tokenized,Status question,Next Event,Next Data,Version ID,Condition 1 Operator,Condition 1 Property,Condition 1 Value
0,conversation_start,"Hi. Welcome to ERHA, \n\nTerima kasih telah me...","['hi', 'welcome', 'to', 'erha', 'terima', 'kas...",True,goto,conversation_start:user_info,NaN,NaN,NaN,NaN
1,fallback_too_short,"Keyword yang dimasukkan tidak Elsa mengerti, s...","['keyword', 'elsa', 'erti', 'silah', 'ketik', ...",True,NaN,NaN,NaN,INCLUDES,source,whatsapp
2,fallback_too_complex,"Keyword yang dimasukkan tidak Elsa mengerti, s...","['keyword', 'elsa', 'erti', 'silah', 'ketik', ...",True,NaN,NaN,NaN,INCLUDES,source,whatsapp
3,fallback_too_complex_simplify,"Keyword yang dimasukkan tidak Elsa mengerti, s...","['keyword', 'elsa', 'erti', 'silah', 'ketik', ...",True,NaN,NaN,NaN,INCLUDES,source,whatsapp
4,handover_livechat_agent_end,Terima kasih sudah menghubungi ERHA,"['terima', 'kasih', 'hubung', 'erha']",True,NaN,NaN,NaN,NaN,NaN,NaN


In [67]:
df2 = df[df['Content ID'].isin(['product_info','treatment_info'])][['Content ID', 'Version ID', 'Answer 1']]
df2

,Content ID,Version ID,Answer 1
50,product_info,erha4,Berikut Informasi mengenai *Moisturizer ERHA 4...
51,product_info,erha5,Berikut Informasi mengenai *Erha 5 Evening Moi...
52,product_info,erha6,Berikut Informasi mengenai *Erha 6 Evening Moi...
54,product_info,erha7,Berikut Informasi mengenai *Erha 7 Foundation*...
55,product_info,erha9,Berikut Informasi mengenai *Erha 9 Foundation*...
...,...,...,...
546,treatment_info,BLACK CARBON SKINPROVEMENT LASER,_Black Carbon Skinprovement Laser_ adalah tind...
547,product_info,Acne Guard Cream,"HISERHA Acne Guard Cream, merupakan krim pelem..."
548,product_info,HISERHA ENERGY BRIGHT CREAM,"HISERHA Energy Bright Cream, merupakan krim pe..."
549,product_info,Urban Skin Defense Multi Benefit Refreshing Spray,ERHA PERFECT SHIELD URBAN SKIN DEFENSE Multi B...


In [68]:
df2['Content ID'].unique()

array(['product_info', 'treatment_info'], dtype=object)

In [69]:
df2['Version ID'] = df2['Version ID'].map(lambda x : str(x).lower())

In [70]:
question_pattern_list = []

for c, q in zip(df2['Content ID'], df2['Version ID']):
    question_pattern_list.append(model.create_questions(str(c), str(q).lower()))

df2['Question Pattern'] = question_pattern_list
df2

,Content ID,Version ID,Answer 1,Question Pattern
50,product_info,erha4,Berikut Informasi mengenai *Moisturizer ERHA 4...,"[erha4, kegunaan erha4, penggunaan erha4, cara..."
51,product_info,erha5,Berikut Informasi mengenai *Erha 5 Evening Moi...,"[erha5, kegunaan erha5, penggunaan erha5, cara..."
52,product_info,erha6,Berikut Informasi mengenai *Erha 6 Evening Moi...,"[erha6, kegunaan erha6, penggunaan erha6, cara..."
54,product_info,erha7,Berikut Informasi mengenai *Erha 7 Foundation*...,"[erha7, kegunaan erha7, penggunaan erha7, cara..."
55,product_info,erha9,Berikut Informasi mengenai *Erha 9 Foundation*...,"[erha9, kegunaan erha9, penggunaan erha9, cara..."
...,...,...,...,...
546,treatment_info,black carbon skinprovement laser,_Black Carbon Skinprovement Laser_ adalah tind...,"[black carbon skinprovement laser, apa itu bla..."
547,product_info,acne guard cream,"HISERHA Acne Guard Cream, merupakan krim pelem...","[acne guard cream, kegunaan acne guard cream, ..."
548,product_info,hiserha energy bright cream,"HISERHA Energy Bright Cream, merupakan krim pe...","[hiserha energy bright cream, kegunaan hiserha..."
549,product_info,urban skin defense multi benefit refreshing spray,ERHA PERFECT SHIELD URBAN SKIN DEFENSE Multi B...,[urban skin defense multi benefit refreshing s...


In [61]:
df2[df2['Version ID'] == 'paket anti aging home peeling \xe2\x80\x93 brightening home peeling solution']['Answer 1']

Series([], Name: Answer 1, dtype: object)

## Creating JSON

In [81]:
def unit_transform(str):
    replacements = {
            '[0-9]+,' : '.',
        }

    for key, value in replacements.items():
        str = re.sub(key, value, str)
    return (str)

In [77]:
def generate_json(df):
    contexts = df['Question Pattern']
    tags = df['Version ID']
    responses = df['Answer 1']
    result = "{\"intents\": ["

    for context, tag, response in zip(contexts, tags, responses):
        result = result + "{\"context\": \"\",\"patterns\": " + str(context) + ",\"responses\": "+ "[\""+ str(response) +"\"]" +",\"tag\": \""+ str(tag) +"\"},"

    result = r"{}".format(result)
    result = result.replace("\'", "\"")
    result = result.replace("\'", "\"")
    # result = result.replace("“", r"\"")
    # result = result.replace("”", r"\"")
    # result = result.replace("–", r"-")

    return unit_transform(result) + "]}"

In [82]:
f = open("data.json", "w")
f.write(str(generate_json(df2).encode('ascii', 'replace')))
f.close()

## Building Model

In [3]:
os.chdir("E:\\Documents\\Coding\\Notebooks\\Chatbot\\apotek")
print(os.getcwd())

E:\Documents\Coding\Notebooks\Chatbot\apotek


In [43]:
data_file = open("data.json").read()
data = json.loads(data_file)

In [44]:
words = []
classes = []
data_X = []
data_y = []

In [45]:
for intent in data["intents"]:
    for pattern in intent["patterns"]:
        tokens = nltk.word_tokenize(pattern)
        words.extend(tokens)
        data_X.append(pattern)
        data_y.append(intent["tag"])
    
    if intent["tag"] not in classes:
        classes.append(intent["tag"])

In [46]:
# create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

In [47]:
# stemming process
sentence = 'Perekonomian Indonesia sedang dalam pertumbuhan yang membanggakan'
output   = stemmer.stem(sentence)

print(output)

ekonomi indonesia sedang dalam tumbuh yang bangga


In [48]:
words = [stemmer.stem(word.lower()) for word in words]
words = sorted(set(words))
classes = sorted(set(classes))

In [49]:
print(words)

['', '-', '01', '10', '100', '15', '2', '20', '20g', '220g', '240', '250', '250g', '270', '30', '30g', '370', '4d', '50', 'abs', 'abs bpo 25', 'abs bpo 5', 'acl', 'acne', 'acne peeling', 'acneact', 'acneact scar treatment by microneedle rf', 'acnic', 'acs', 'acs1', 'acs2', 'acs3', 'acs4', 'acsbp', 'acsm', 'activator', 'active', 'advance', 'aef', 'aesthefill', 'af', 'af1', 'af2', 'af3', 'af4', 'af5', 'afp', 'aft', 'aftss', 'ag2', 'age', 'aging', 'ainf1', 'al', 'al1', 'al2', 'alcl', 'alcl3', 'alcohol', 'all', 'allantoin', 'allderma', 'am', 'amg', 'amg1', 'amg10', 'amg2', 'amg3', 'amg4', 'amg5', 'amg6', 'amg7', 'amg8', 'amg9', 'anc', 'anc1', 'anc2', 'anc3', 'anc4', 'anc5', 'and', 'anti', 'apa', 'app', 'aqua', 'aselo', 'asf', 'asg', 'asm', 'asmd', 'ast', 'at', 'at 5', 'at1', 'at2', 'at3', 'away', 'aza', 'b-energetic', 'baby', 'back', 'bagaimana', 'balm', 'barrier', 'basic', 'bb', 'beard', 'bedah electrocauter', 'beige', 'benefit', 'berapa', 'bha', 'biaya', 'bkc', 'black', 'blemish', 'blue 

In [ ]:
print(data_X)

In [51]:
training = []
out_empty = [0] * len(classes)

In [52]:
for idx, doc in enumerate(data_X):
    bow = []
    text = stemmer.stem(doc.lower())
    for word in words:
        bow.append(1) if word in text else bow.append(0)
        
    output_row = list(out_empty)
    output_row[classes.index(data_y[idx])] = 1
    
    training.append([bow, output_row])

In [53]:
random.shuffle(training)
training = np.array(training, dtype=object)

train_X = np.array(list(training[:,0]))
train_Y = np.array(list(training[:,1]))

In [54]:
print(train_Y[11])

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [55]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_X[0]),), activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(len(train_Y[0]), activation="softmax"))

adam = tf.keras.optimizers.Adam(learning_rate=0.01, decay=1e-6)
model.compile(loss="categorical_crossentropy", optimizer=adam, metrics=["accuracy"])

print(model.summary())
model.fit(x=train_X, y=train_Y, epochs=150, verbose=1)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 128)               84608     
                                                                 
 dropout_2 (Dropout)         (None, 128)               0         
                                                                 
 dense_4 (Dense)             (None, 64)                8256      
                                                                 
 dropout_3 (Dropout)         (None, 64)                0         
                                                                 
 dense_5 (Dense)             (None, 443)               28795     
                                                                 
Total params: 121,659
Trainable params: 121,659
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/150
168/168 [======================

In [56]:
def clean_text(text):
    tokens = nltk.word_tokenize(text)
    tokens = [stemmer.stem(word) for word in tokens]
    return tokens

In [57]:
def bag_of_words(text, vocab):
    tokens = clean_text(text)
    bow = [0] * len(vocab)
    for w in tokens:
        for idx, word in enumerate(vocab):
            if word == w:
                bow[idx] = 1
    return np.array(bow)

In [58]:
def pred_class(text, vocab, labels):
    bow = bag_of_words(text, vocab)
    result = model.predict(np.array([bow]))[0]
    thresh = 0.5
    y_pred = [[indx, res] for indx, res in enumerate(result) if res > thresh]
    y_pred.sort(key=lambda x: x[1], reverse=True)
    
    return_list = []
    for r in y_pred:
        return_list.append(labels[r[0]])
    
    return return_list

In [59]:
def get_response(intents_list, intents_json):
    if len(intents_list) == 0:
        result = "Sorry! I don't understand"
    else:
        tag = intents_list[0]
        list_of_intents = intents_json["intents"]
        
        for i in list_of_intents:
            if i["tag"] == tag:
                result = random.choice(i["responses"])
                break
    return result

In [70]:
message = "apa sih acl itu"
intents = pred_class(message, words, classes)
result = get_response(intents, data)
print(result)

1/1 [==============================] - 0s 35ms/step
Berikut Informasi mengenai *ANTI COMEDONAL LOTION*

Kegunaan : 
Lotion yang digunakan untuk mengurangi dan melunakkan komedo

Ukuran :
30 ml

Cara Pakai : 
Pada saat malam hari dengan cara bersihkan wajah terlebih dahulu, kemudian oleskan ACL secara merata pada area yang berkomedo, diamkan selama kurang lebih 5 ? 10 menit. Selanjutnya cuci muka menggunakan air mengalir.

Cara Penyimpanan : 
Pada suhu dingin 10 - 15/ 15-20 C
Disimpan di cooler/kulkas pada bagian pintu


In [62]:
print(words)

['', '-', '01', '10', '100', '15', '2', '20', '20g', '220g', '240', '250', '250g', '270', '30', '30g', '370', '4d', '50', 'abs', 'abs bpo 25', 'abs bpo 5', 'acl', 'acne', 'acne peeling', 'acneact', 'acneact scar treatment by microneedle rf', 'acnic', 'acs', 'acs1', 'acs2', 'acs3', 'acs4', 'acsbp', 'acsm', 'activator', 'active', 'advance', 'aef', 'aesthefill', 'af', 'af1', 'af2', 'af3', 'af4', 'af5', 'afp', 'aft', 'aftss', 'ag2', 'age', 'aging', 'ainf1', 'al', 'al1', 'al2', 'alcl', 'alcl3', 'alcohol', 'all', 'allantoin', 'allderma', 'am', 'amg', 'amg1', 'amg10', 'amg2', 'amg3', 'amg4', 'amg5', 'amg6', 'amg7', 'amg8', 'amg9', 'anc', 'anc1', 'anc2', 'anc3', 'anc4', 'anc5', 'and', 'anti', 'apa', 'app', 'aqua', 'aselo', 'asf', 'asg', 'asm', 'asmd', 'ast', 'at', 'at 5', 'at1', 'at2', 'at3', 'away', 'aza', 'b-energetic', 'baby', 'back', 'bagaimana', 'balm', 'barrier', 'basic', 'bb', 'beard', 'bedah electrocauter', 'beige', 'benefit', 'berapa', 'bha', 'biaya', 'bkc', 'black', 'blemish', 'blue 